In [3]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

from shapely import geometry
from shapely.ops import unary_union
from polygeohasher import polygeohasher
import geohash

In [4]:
df_geoh_stops = pd.read_csv('../data/activity_2023-04-01_2024-03-31.csv')
gdf_met = gpd.read_file('../data/metro_regions_raw.geojson')

MERGE_REGIONS = [
    "Ottawa - Gatineau (Ontario part / partie de l'Ontario) (B)", 
    'Ottawa - Gatineau (partie du Québec / Quebec part) (B)',
]

ottawa_poly = unary_union(gdf_met[gdf_met['name'].isin(MERGE_REGIONS)].geometry.to_list())
ottawa_pop = gdf_met[gdf_met['name'].isin(MERGE_REGIONS)]['population'].sum()
row = {'name': ['Ottawa - Gatineau (B)'], 'population': [ottawa_pop], 'geometry': [ottawa_poly]}
gdf_met = gpd.GeoDataFrame(pd.concat([gdf_met, gpd.GeoDataFrame(row, crs=gdf_met.crs)]))

gdf_met = gdf_met.drop(gdf_met[gdf_met.name.isin(MERGE_REGIONS)].index).sort_values(by='population', ascending=False).reset_index(drop=True)
gdf_met

,name,population,geometry
0,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",19908595.0,"MULTIPOLYGON (((-72.03683 41.24984, -72.03496 ..."
1,"Los Angeles-Long Beach-Anaheim, CA Metro Area",13111917.0,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987..."
2,"Chicago-Naperville-Elgin, IL-IN-WI Metro Area",9566955.0,"MULTIPOLYGON (((-88.94215 42.06505, -88.93894 ..."
3,"Dallas-Fort Worth-Arlington, TX Metro Area",7673379.0,"MULTIPOLYGON (((-98.06560 32.59502, -98.06486 ..."
4,"Houston-The Woodlands-Sugar Land, TX Metro Area",7142603.0,"MULTIPOLYGON (((-94.71830 29.72885, -94.71721 ..."
...,...,...,...
295,"Sebastian-Vero Beach, FL Metro Area",160986.0,"MULTIPOLYGON (((-80.88157 27.82251, -80.86888 ..."
296,"Jackson, MI Metro Area",160637.0,"MULTIPOLYGON (((-84.71860 42.33454, -84.71849 ..."
297,"State College, PA Metro Area",158665.0,"MULTIPOLYGON (((-78.37688 40.75254, -78.37652 ..."
298,"Idaho Falls, ID Metro Area",158478.0,"MULTIPOLYGON (((-113.79814 43.57197, -113.7958..."


Fix region namings and compute centroids

In [8]:
CITY_TO_PROVINCE = {
    'Toronto': 'ON',
    'Montreal': 'QC',
    'Vancouver': 'BC',
    'Ottawa - Gatineau': 'ON-QC',
    'Calgary': 'AB',
    'Edmonton': 'AB',
    'Quebec': 'QC',
    'Winnipeg': 'MB',
    'Hamilton': 'ON',
    'Kitchener - Cambridge - Waterloo': 'ON',
    'London': 'ON',
    'Halifax': 'NS',
    'St. Catharines - Niagara': 'ON',
    'Windsor': 'ON',
    'Oshawa': 'ON',
    'Victoria': 'BC',
    'Saskatoon': 'SK',
    'Regina': 'SK',
    'Sherbrooke': 'QC',
    'Kelowna': 'BC',
    'Barrie': 'ON',
    "St. John's": 'NL',
    'Abbotsford - Mission': 'BC',
    'Kingston': 'ON',
    'Greater Sudbury / Grand Sudbury': 'ON',
    'Guelph': 'ON',
    'Saguenay': 'QC',
    'Trois-Rivieres': 'QC',
    'Moncton': 'NB',
}

In [6]:
def clean_region_name(region):
    region = region.replace(" Metro Area", "")
    region = region.replace(" (B)", "")

    # Accented normalization
    if 'è' in region:
        region = region.replace('è', 'e')
    if 'é' in region:
        region = region.replace('é', 'e')
    
    # Canadian province codes
    if "," not in region:  
        region = f"{region}, {CITY_TO_PROVINCE[region]}"

    # Louisville and Greater Sudbury
    if '/' in region:
        s = region.split(',')
        s[0] = s[0].split('/')[0].strip()
        region = ",".join(s)
    
    return region

In [11]:
gdf_met['name'] = gdf_met.apply(lambda row: clean_region_name(row['name']), axis=1)
gdf_met.to_file('../data/metro_regions_full.geojson', driver='GeoJSON')
gdf_met.to_file('../src/data/metro_regions_full.geo.json', driver='GeoJSON')

In [13]:
gdf_cent = gdf_met.set_geometry(gdf_met.centroid)
gdf_cent.to_file('../src/data/metro_regions_centroids.geo.json', driver='GeoJSON')  # Fix these manually

/tmp/ipykernel_217884/3717172684.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_cent = gdf_met.set_geometry(gdf_met.centroid)


Generate geohashes for each metro region

In [3]:
pgh = polygeohasher.Polygeohasher(gdf_met)

In [4]:
INPUT_GEOHASH_LEVEL = 6
df_init = pgh.create_geohash_list(INPUT_GEOHASH_LEVEL, inner=False)  # 10 mins

In [5]:
gdf_geo = pgh.geohashes_to_geometry(df_init, "geohash_list")  # 2-3 mins

In [6]:
def compute_intersect_area(gdf, row):
    """ Given the region polygons (gdf) and the geohash row (row), return the
    area of the intersection. 
    """
    region_name = row['name']
    region_poly = gdf.loc[gdf['name'] == region_name]['geometry'].values[0]
    return row['geometry'].intersection(region_poly).area

In [7]:
gdf_geo['intersect_area'] = gdf_geo.progress_apply(lambda x: compute_intersect_area(gdf_met, x), axis=1)  # 30 mins

100%|███████████████████████████████| 4017916/4017916 [29:15<00:00, 2288.39it/s]


In [8]:
gdf_geo = gdf_geo.sort_values(by=['intersect_area'], ascending=False).drop_duplicates(subset=['geohash_list'])
gdf_geo = gdf_geo.sort_values(by=['name', 'geohash_list'])
gdf_geo = gdf_geo.drop(['intersect_area', 'population'], axis=1)
gdf_geo = gdf_geo.rename(columns={"geohash_list": "geohash"})
df_geo = pd.DataFrame(gdf_geo.drop(columns='geometry'))

In [9]:
df_geo.to_csv('../data/metro_region_geohashes.csv', index=False)

Link the activity data to the metro regions by geohash

In [4]:
def geohash_to_polygon(geo):
    """
    https://github.com/rohitsinghsalyan/polygeohasher/blob/master/polygeohasher/polygon_geohash_convertor.py
    :param geo: String that represents the geohash.
    :return: Returns a Shapely's Polygon instance that represents the geohash.
    """
    lat_centroid, lng_centroid, lat_offset, lng_offset = geohash.decode_exactly(geo)

    corner_1 = (lat_centroid - lat_offset, lng_centroid - lng_offset)[::-1]
    corner_2 = (lat_centroid - lat_offset, lng_centroid + lng_offset)[::-1]
    corner_3 = (lat_centroid + lat_offset, lng_centroid + lng_offset)[::-1]
    corner_4 = (lat_centroid + lat_offset, lng_centroid - lng_offset)[::-1]

    return geometry.Polygon([corner_1, corner_2, corner_3, corner_4, corner_1])

In [5]:
df_geo = pd.read_csv('../data/metro_region_geohashes.csv')

In [7]:
df_geoh_stops = df_geoh_stops.rename(columns={"geohash6": "geohash"})
df_region_geo_stops = pd.merge(df_geo, df_geoh_stops, on="geohash")

In [10]:
total_stops = df_region_geo_stops['total_stops'].sum()

In [ ]:
metro_regions = gdf_met['name'].unique()
for region in tqdm(metro_regions):
    df_subset = df_region_geo_stops[df_region_geo_stops['name'] == region]
    df_subset = df_subset.drop(columns=['name'])

    subset_stops = df_subset['total_stops'].sum()
    df_subset['prop_total_stops'] = df_subset['total_stops'] / total_stops
    df_subset['prop_subset_stops'] = df_subset['total_stops'] / subset_stops
    df_subset = df_subset.drop(columns=['total_stops'])
    
    # # Unified Ottawa region only
    # if region in EXCLUDE_REGIONS:
    #     continue

    # cur_region = clean_region_name(region)

    df_subset['geometry'] = df_subset.apply(lambda x: geohash_to_polygon(x['geohash']), axis=1)
    gdf_subset = gpd.GeoDataFrame(df_subset)

    gdf_subset.to_file(f'../data/metro_region_geohash_stops/{cur_region}.geojson', driver='GeoJSON')

 92%|█████████████████████████████████████▊   | 271/294 [08:39<00:26,  1.15s/it]